# init

In [47]:
import pandas, IPython.core.display

## open file

In [48]:
src = pandas.ExcelFile('data/real/2017/CAM data from iPads/2017 CAM data from iPads.xlsx')

In [49]:
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [50]:
sheets_to_process = {sheet_name.split(' ')[-1]: src.parse(sheet_name) for sheet_name in src.sheet_names
    if any([person_name in sheet_name for person_name in ['Erl', 'Tyler']])}

# rename columns

In [51]:
sheets_to_rename = [sheet for sheet in sheets_to_process.values()
                    if any([str(column).find(' ') for column in sheet.columns])]

In [52]:
for sheet_to_rename in sheets_to_rename:
    sheet_to_rename.rename(mapper=lambda x: str(x).split(' ')[0], axis='columns', inplace=True)

# concatenate sheets

In [53]:
df = pandas.concat(sheets_to_process, names=['Sheet','Row'])

# generate hierarchical index (Python)

In [54]:
import collections

In [55]:
class NestedDefaultdict(collections.defaultdict):
    data = None
    name = None
    parent_key = None

    def __init__(self):
        collections.defaultdict.__init__(self, NestedDefaultdict)

In [56]:
def get_column_tree(frame):
    coltree = NestedDefaultdict()
    for column in frame.columns:
        pointer = coltree
        pointer.parent_key = ''
        for word in str(column).split('__'):
            pointer = pointer[word]
            pointer.parent_key = word
        pointer.name = column
        pointer.data = frame[column]
    coltree.default_factory = None
    return coltree

In [57]:
coltree = get_column_tree(df)

## visualize

In [58]:
def visualize_tree(node):
    """Given a dictionary, display a list of keys, and all keys 
    of any contained dictionaries, recursively. Indent to show depth
    of recursion."""

    def visualize_node(node, depth=0):
        for key, value in node.items():
            output.append(f"{'.   ' * depth}{key}"
                          f"{'/' if value.keys() else ''}"
                          f"{' : ' + str(value.data.count()) if value.data is not None else ''}"
                         )
            visualize_node(value, depth + 1)

    output = []
    visualize_node(node)
    return '\n'.join(output)

In [59]:
print(visualize_tree(coltree))

clients/
.   company : 1
.   displayText : 1
.   fname : 1
.   lname : 1
.   name : 1
fields/
.   client/
.   .   company : 21
.   .   displayText : 21
.   .   fname : 21
.   .   lname : 21
.   .   name : 21
.   crop : 21
.   date : 21
.   desc : 15
.   image : 21
.   name : 21
.   oSets/
.   .   completeSets : 54
.   .   date : 54
.   .   dateCompare : 54
.   .   desc : 0
.   .   growthStage : 54
.   .   oPoints/
.   .   .   id : 335
.   .   .   location/
.   .   .   .   coords/
.   .   .   .   .   accuracy : 250
.   .   .   .   .   altitude : 250
.   .   .   .   .   altitudeAccuracy : 250
.   .   .   .   .   heading : 247
.   .   .   .   .   latitude : 250
.   .   .   .   .   longitude : 250
.   .   .   .   .   speed : 250
.   .   .   .   timestamp : 250
.   .   .   name : 335
.   .   .   observations/
.   .   .   .   a1/
.   .   .   .   .   number : 448
.   .   .   .   a2/
.   .   .   .   .   number : 142
.   .   .   .   a3/
.   .   .   .   .   number : 25
.   .   .   .   anum : 167

# define functions: get nodes, column names, DataFrames

In [60]:
def get_child_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        if child.name is not None:
            return_value.append(child)
    return return_value

In [61]:
def get_descendant_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        return_value.extend(get_descendant_columns(child))
    return return_value

In [62]:
def get_child_column_names(node):
    return [column.name for column in get_child_columns(node)]

In [63]:
def get_descendant_column_names(node):
    return [column.name for column in get_descendant_columns(node)]

In [64]:
def get_child_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_child_columns(node)})
    return frame[get_child_column_names(node)]

In [65]:
def get_descendant_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_descendant_columns(node)})
    return frame[get_descendant_column_names(node)]

# convert_datetime

In [66]:
def convert_datetime(frame):
    output_frame = frame.copy()
    
    foplt = 'fields__oSets__oPoints__location__timestamp'
    output_frame[foplt] = pandas.to_datetime(frame[foplt], unit='ms')
    
    for column in ['fields__oSets__date', 'fields__date']:
        output_frame[column] = pandas.to_datetime(output_frame[column], infer_datetime_format=True)
    
    return output_frame

In [67]:
df = convert_datetime(df)

# cleanup

In [68]:
df2 = df.copy().reset_index()

In [69]:
df2.columns

Index(['Sheet', 'Row', 'clients__company', 'clients__displayText',
       'clients__fname', 'clients__lname', 'clients__name',
       'fields__client__company', 'fields__client__displayText',
       'fields__client__fname', 'fields__client__lname',
       'fields__client__name', 'fields__crop', 'fields__date', 'fields__desc',
       'fields__image', 'fields__name', 'fields__oSets__completeSets',
       'fields__oSets__date', 'fields__oSets__dateCompare',
       'fields__oSets__desc', 'fields__oSets__growthStage',
       'fields__oSets__oPoints__id',
       'fields__oSets__oPoints__location__coords__accuracy',
       'fields__oSets__oPoints__location__coords__altitude',
       'fields__oSets__oPoints__location__coords__altitudeAccuracy',
       'fields__oSets__oPoints__location__coords__heading',
       'fields__oSets__oPoints__location__coords__latitude',
       'fields__oSets__oPoints__location__coords__longitude',
       'fields__oSets__oPoints__location__coords__speed',
       'fiel

In [70]:
for column in (
    'Sheet', 'Row',
    'clients__company',
    'clients__displayText',
    'clients__fname',
    'clients__lname',
    'clients__name',
    'fields__client__company',
#     'fields__client__displayText',
    'fields__client__fname',
    'fields__client__lname',
    'fields__client__name', 
#     'fields__crop',
    'fields__date',
#     'fields__desc',
    'fields__image',
#     'fields__name',
#     'fields__oSets__completeSets',
#     'fields__oSets__date', 
    'fields__oSets__dateCompare',
#     'fields__oSets__desc',
#     'fields__oSets__growthStage',
#     'fields__oSets__oPoints__id',
    'fields__oSets__oPoints__location__coords__accuracy',
    'fields__oSets__oPoints__location__coords__altitude',
    'fields__oSets__oPoints__location__coords__altitudeAccuracy',
    'fields__oSets__oPoints__location__coords__heading',
    'fields__oSets__oPoints__location__coords__latitude',
    'fields__oSets__oPoints__location__coords__longitude',
    'fields__oSets__oPoints__location__coords__speed',
    'fields__oSets__oPoints__location__timestamp',
    'fields__oSets__oPoints__name',
#     'fields__oSets__oPoints__observations__a1__number',
#     'fields__oSets__oPoints__observations__a2__number',
#     'fields__oSets__oPoints__observations__a3__number',
#     'fields__oSets__oPoints__observations__anum',
    'fields__oSets__oPoints__observations__complete',
#     'fields__oSets__oPoints__observations__disabled',
    'fields__oSets__oPoints__observations__eVnum',
#     'fields__oSets__oPoints__observations__enum',
    'fields__oSets__oPoints__observations__id',
#     'fields__oSets__oPoints__observations__name',
#     'fields__oSets__oPoints__observations__|',
#     'fields__oSets__oPoints__observations__|__number',
#     'fields__oSets__obsName',
    'fields__oSets__results',
#     'fields__oSets__totalA1', 'fields__oSets__totalA2',
#     'fields__oSets__totalA3', 'fields__oSets__totalA4',
#     'fields__oSets__totalSets',
    'observers'
):
    del df2[column]

In [71]:
df2.columns

Index(['fields__client__displayText', 'fields__crop', 'fields__desc',
       'fields__name', 'fields__oSets__completeSets', 'fields__oSets__date',
       'fields__oSets__desc', 'fields__oSets__growthStage',
       'fields__oSets__oPoints__id',
       'fields__oSets__oPoints__observations__a1__number',
       'fields__oSets__oPoints__observations__a2__number',
       'fields__oSets__oPoints__observations__a3__number',
       'fields__oSets__oPoints__observations__anum',
       'fields__oSets__oPoints__observations__disabled',
       'fields__oSets__oPoints__observations__enum',
       'fields__oSets__oPoints__observations__name',
       'fields__oSets__oPoints__observations__|',
       'fields__oSets__oPoints__observations__|__number',
       'fields__oSets__obsName', 'fields__oSets__totalA1',
       'fields__oSets__totalA2', 'fields__oSets__totalA3',
       'fields__oSets__totalA4', 'fields__oSets__totalSets'],
      dtype='object')

In [72]:
def category_columns():
    """
    for column in (
    'fields__client__displayText',
    'fields__crop',
    'fields__desc',
    'fields__name',
    #     'fields__oSets__completeSets',
    #     'fields__oSets__date',
    'fields__oSets__desc',
    'fields__oSets__growthStage',
    'fields__oSets__oPoints__id',
    #     'fields__oSets__oPoints__location__timestamp',
    #     'fields__oSets__oPoints__name',
    #     'fields__oSets__oPoints__observations__a1__number',
    #     'fields__oSets__oPoints__observations__a2__number',
    #     'fields__oSets__oPoints__observations__a3__number',
    #     'fields__oSets__oPoints__observations__anum',
    'fields__oSets__oPoints__observations__disabled',
    #     'fields__oSets__oPoints__observations__enum',
#     'fields__oSets__oPoints__observations__id',
    'fields__oSets__oPoints__observations__name',
    'fields__oSets__oPoints__observations__|',
#     'fields__oSets__oPoints__observations__|__number',
    'fields__oSets__obsName',
    #     'fields__oSets__totalA1',
    #     'fields__oSets__totalA2',
    #     'fields__oSets__totalA3',
    #     'fields__oSets__totalA4',
    #     'fields__oSets__totalSets'
):
    df2[column] = df2[column].astype('category')
"""
    pass

In [73]:
tree = get_column_tree(df2)

# fields

In [74]:
fields_node = tree['fields']
fields_columns = get_child_column_names(fields_node) + get_descendant_column_names(fields_node['client'])
fields = df2[fields_columns].dropna(how='all').sort_values('fields__name')
fields.set_index('fields__crop').sort_index()

,fields__desc,fields__name,fields__client__displayText
fields__crop,,,
CROPS.BARLEY,Right next to Alvena Canola,Alvena Barley,"AAFC SRDC, Tyler Wist"
CROPS.BARLEY,Right next to SEF wheat,SEF Barley,"AAFC SRDC, Tyler Wist"
CROPS.BARLEY,Next to SEF Wheat and near faba beans,SEF Barley,"AAFC SRDC, Tyler Wist"
CROPS.BARLEY,NaN,Llewelyn Barley,"AAFC SRDC, Tyler Wist"
CROPS.OATS,Next to SEF peas and a wheat field,SEF Oats,"AAFC SRDC, Tyler Wist"
CROPS.OATS,"Next to SEF Peas, near the canola",SEF Oats,"AAFC SRDC, Tyler Wist"
CROPS.WHEAT,Next to soybeans and canola,Outlook wheat 1,"AAFC SRDC, Tyler Wist"
CROPS.WHEAT,Wheat 2 by canola and soybeans but on other si...,Outlook wheat,"AAFC SRDC, Tyler Wist"
CROPS.WHEAT,NaN,Outlook Wheat-2,"AAFC SRDC, Tyler Wist"


# sets

In [75]:
sets_node = fields_node['oSets']
sets_columns = get_child_column_names(sets_node)
sets = df2[sets_columns].dropna(how='all')
sets

,fields__oSets__completeSets,fields__oSets__date,fields__oSets__desc,fields__oSets__growthStage,fields__oSets__obsName,fields__oSets__totalA1,fields__oSets__totalA2,fields__oSets__totalA3,fields__oSets__totalA4,fields__oSets__totalSets
0,0.0,2017-08-02 13:12:09.542,NaN,7.0,Tyler,NaN,NaN,NaN,NaN,1.0
70,1.0,2017-08-09 09:25:11.710,NaN,8.0,Tyler,164.0,0.0,0.0,0.0,1.0
140,1.0,2017-08-09 10:06:25.480,NaN,7.0,Tyler,66.0,0.0,0.0,0.0,1.0
210,2.0,2017-08-09 11:21:01.555,NaN,9.0,Stean,0.0,0.0,0.0,0.0,2.0
350,2.0,2017-08-09 11:37:20.862,NaN,8.0,Stean,5.0,5.0,0.0,0.0,2.0
490,1.0,2017-08-22 15:42:05.751,NaN,8.0,Mikki,1.0,0.0,0.0,0.0,1.0
560,2.0,2017-08-17 11:12:02.820,NaN,8.0,Gabrielle,169.0,96.0,0.0,0.0,2.0
700,2.0,2017-08-17 13:06:30.183,NaN,9.0,Stean,78.0,102.0,0.0,0.0,2.0
840,1.0,2017-08-22 16:02:50.682,NaN,8.0,Mikki,187.0,0.0,0.0,0.0,1.0
910,0.0,2017-07-14 12:31:24.194,NaN,6.0,Tyler,NaN,NaN,NaN,NaN,1.0


# points

In [76]:
points_node = sets_node['oPoints']
print(visualize_tree(points_node))

id : 335
observations/
.   a1/
.   .   number : 448
.   a2/
.   .   number : 142
.   a3/
.   .   number : 25
.   anum : 1675
.   disabled : 0
.   enum : 335
.   name : 2010
.   |/ : 3015
.   .   number : 172


In [77]:
points_columns = get_child_column_names(points_node) + get_child_column_names(points_node['location'])
points = df2[points_columns].dropna(how='all')
points

,fields__oSets__oPoints__id
0,0.0
14,1.0
28,2.0
42,3.0
56,4.0
70,0.0
84,1.0
98,2.0
112,3.0
126,4.0


# observations

In [78]:
observation_node = points_node['observations']
print(visualize_tree(observation_node))

a1/
.   number : 448
a2/
.   number : 142
a3/
.   number : 25
anum : 1675
disabled : 0
enum : 335
name : 2010
|/ : 3015
.   number : 172


## cleanup

In [95]:
idxcols = {'fields__oSets__date':                               'datetime',
           'fields__oSets__oPoints__id':                        'point_id',
           'fields__oSets__oPoints__observations__name':        'observations_name',
           'fields__oSets__oPoints__observations__|':           'natural_enemy_name',
           'fields__oSets__oPoints__observations__|__number':   'natural_enemy_count',
           }
idf = df2.rename(columns=idxcols)
for column in ['datetime', 'point_id', 'observations_name']:
    idf[column] = idf[column].ffill()

### split string to columns

In [96]:
new_columns = idf.pop('observations_name').str.extract('(?P<observation_subject>Aphid|Natural Enemy) Observation ?(?P<observation_id>\d+)?', expand=True)
idf = idf.join(new_columns)

In [97]:
idf = idf.set_index(['datetime', 'point_id','observation_id','observation_subject'])

In [98]:
idf

fields__client__displayText  \
datetime                point_id observation_id observation_subject                               
2017-08-02 13:12:09.542 0.0      1              Aphid                                       NaN   
                                 2              Aphid                                       NaN   
                                 3              Aphid                                       NaN   
                                 4              Aphid                                       NaN   
                                 5              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        1.0      6              Aphid                                       NaN   
                                 7              Aphid                                       NaN   
                                 8              Aphid                                       NaN   
                                 9              Aphid                                       NaN   
                                 10             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        2.0      11             Aphid                                       NaN   
                                 12             Aphid                                       NaN   
...                                                                                         ...   
2017-08-22 15:42:36.395 2.0      NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        3.0      16             Aphid                                       NaN   
                                 17             Aphid                                       NaN   
                                 18             Aphid                                       NaN   
                                 19             Aphid                                       NaN   
                                 20             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
         

### unstack natural enemies

In [99]:
ne = idf.xs('Natural Enemy', level='observation_subject', drop_level=False)

In [100]:
ne[['natural_enemy_name','natural_enemy_count']]

natural_enemy_name  \
datetime                point_id observation_id observation_subject                      
2017-08-02 13:12:09.542 0.0      NaN            Natural Enemy                       e1   
                                                Natural Enemy                       e2   
                                                Natural Enemy                       e3   
                                                Natural Enemy                       e4   
                                                Natural Enemy                       e5   
                                                Natural Enemy                       e6   
                                                Natural Enemy                       e7   
                                                Natural Enemy                       e8   
                                                Natural Enemy                       e9   
                        1.0      NaN            Natural Enemy                       e1   
                                                Natural Enemy                       e2   
                                                Natural Enemy                       e3   
                                                Natural Enemy                       e4   
                                                Natural Enemy                       e5   
                                                Natural Enemy                       e6   
                                                Natural Enemy                       e7   
                                                Natural Enemy                       e8   
                                                Natural Enemy                       e9   
                        2.0      NaN            Natural Enemy                       e1   
                                                Natural Enemy                       e2   
                                                Natural Enemy                       e3   
                                                Natural Enemy                       e4   
                                                Natural Enemy                       e5   
                                                Natural Enemy                       e6   
                                                Natural Enemy                       e7   
                                                Natural Enemy                       e8   
                                                Natural Enemy                       e9   
                        3.0      NaN            Natural Enemy                       e1   
                                                Natural Enemy                       e2   
                                                Natural Enemy                       e3   
...                                                                                ...   
2017-08-22 15:42:36.395 1.0      NaN            Natural Enemy                       e7   
                                                Natural Enemy                       e8   
                                                Natural Enemy                       e9   
                        2.0      NaN            Natural Enemy                       e1   
                                                Natural Enemy                       e2   
                                                Natural Enemy                       e3   
                                                Natural Enemy                       e4   
                                                Natural Enemy                       e5   
                                                Natural Enemy                       e6   
                                                Natural Enemy                       e7   
                                                Natural Enemy                       e8   
                                                Natural Enemy                       e9   
                  

In [101]:
ne.set_index('natural_enemy_name', append=True, inplace=True)

In [103]:
ne

fields__client__displayText  \
datetime                point_id observation_id observation_subject natural_enemy_name                               
2017-08-02 13:12:09.542 0.0      NaN            Natural Enemy       e1                                         NaN   
                                                                    e2                                         NaN   
                                                                    e3                                         NaN   
                                                                    e4                                         NaN   
                                                                    e5                                         NaN   
                                                                    e6                                         NaN   
                                                                    e7                                         NaN   
                                                                    e8                                         NaN   
                                                                    e9                                         NaN   
                        1.0      NaN            Natural Enemy       e1                                         NaN   
                                                                    e2                                         NaN   
                                                                    e3                                         NaN   
                                                                    e4                                         NaN   
                                                                    e5                                         NaN   
                                                                    e6                                         NaN   
                                                                    e7                                         NaN   
                                                                    e8                                         NaN   
                                                                    e9                                         NaN   
                        2.0      NaN            Natural Enemy       e1                                         NaN   
                                                                    e2                                         NaN   
                                                                    e3                                         NaN   
                                                                    e4                                         NaN   
                                                                    e5                                         NaN   
                                                                    e6                                         NaN   
                                                                    e7                                         NaN   
                                                                    e8                                         NaN   
                                                                    e9                                         NaN   
                        3.0      NaN            Natural Enemy       e1                                         NaN   
                                                                    e2                                         NaN   
                                                                    e3                                         NaN   
...                                                                                                            ...   
2017-08-22 15:42:36.395 1.0      NaN            Natural Enemy       e7                                         NaN   
                                                                    e8     

In [104]:
ne2 = ne[['natural_enemy_count']].unstack()

In [105]:
ne2.columns

MultiIndex(levels=[['natural_enemy_count'], ['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8]],
           names=[None, 'natural_enemy_name'])

In [106]:
ne2.columns = ne2.columns.droplevel(level=0)

In [107]:
ne2.columns

Index(['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9'], dtype='object', name='natural_enemy_name')

In [108]:
idf

fields__client__displayText  \
datetime                point_id observation_id observation_subject                               
2017-08-02 13:12:09.542 0.0      1              Aphid                                       NaN   
                                 2              Aphid                                       NaN   
                                 3              Aphid                                       NaN   
                                 4              Aphid                                       NaN   
                                 5              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        1.0      6              Aphid                                       NaN   
                                 7              Aphid                                       NaN   
                                 8              Aphid                                       NaN   
                                 9              Aphid                                       NaN   
                                 10             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        2.0      11             Aphid                                       NaN   
                                 12             Aphid                                       NaN   
...                                                                                         ...   
2017-08-22 15:42:36.395 2.0      NaN            Natural Enemy                               NaN   
                                                Natural Enemy                               NaN   
                        3.0      16             Aphid                                       NaN   
                                 17             Aphid                                       NaN   
                                 18             Aphid                                       NaN   
                                 19             Aphid                                       NaN   
                                 20             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
         

In [40]:
# idf.index = idf.index.droplevel(level='observation_subject')

In [109]:
natural_enemy_column_names = [column_name for column_name in idf.columns if 'natural_enemy_' in column_name]
for column_name in idf[natural_enemy_column_names]:
    del idf[column_name]

In [110]:
index_columns = ['datetime', 'point_id', 'observation_id','observation_subject']
idf2 = idf.reset_index().drop_duplicates(subset=index_columns).set_index(index_columns)

### merge unstacked natural enemy counts into main frame

In [113]:
ne2.sort_index()

natural_enemy_name                                                    e1   e2  \
datetime                point_id observation_id observation_subject             
2017-07-14 12:31:24.194 0.0      NaN            Natural Enemy        1.0  1.0   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-07-18 10:31:22.263 0.0      NaN            Natural Enemy        NaN  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-07-18 10:51:30.195 0.0      NaN            Natural Enemy        NaN  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-07-18 11:39:46.221 0.0      NaN            Natural Enemy        NaN  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-07-18 14:25:46.769 0.0      NaN            Natural Enemy        NaN  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-07-18 14:49:08.831 0.0      NaN            Natural Enemy        NaN  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
...                                                                  ...  ...   
2017-08-24 13:10:20.476 5.0      NaN            Natural Enemy        NaN  NaN   
                        6.0      NaN            Natural Enemy        NaN  NaN   
                        7.0      NaN            Natural Enemy        NaN  NaN   
                        8.0      NaN            Natural Enemy        NaN  NaN   
                        9.0      NaN            Natural Enemy        NaN  NaN   
2017-08-24 16:57:43.540 0.0      NaN            Natural Enemy        1.0  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        NaN  NaN   
                        3.0      NaN            Natural Enemy        NaN  NaN   
                        4.0      NaN            Natural Enemy        NaN  NaN   
2017-08-25 12:54:07.934 0.0      NaN            Natural Enemy        4.0  NaN   
                        1.0      NaN            Natural Enemy        NaN  NaN   
                        2.0      NaN            Natural Enemy        2.0  3.0   
                        3.0      NaN            Natural Enemy        4.0  1.0   
                        4.0      NaN            Natural Enemy        2.0  NaN   
                        5.0      NaN            Natural Enemy        2.0  NaN   
                        6.0    

In [114]:
idf2.sort_index()

fields__client__displayText  \
datetime                point_id observation_id observation_subject                               
2017-07-14 12:31:24.194 0.0      1              Aphid                     AAFC SRDC, Tyler Wist   
                                 2              Aphid                                       NaN   
                                 3              Aphid                                       NaN   
                                 4              Aphid                                       NaN   
                                 5              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        1.0      10             Aphid                                       NaN   
                                 6              Aphid                                       NaN   
                                 7              Aphid                                       NaN   
                                 8              Aphid                                       NaN   
                                 9              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        2.0      11             Aphid                                       NaN   
                                 12             Aphid                                       NaN   
                                 13             Aphid                                       NaN   
                                 14             Aphid                                       NaN   
                                 15             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        3.0      16             Aphid                                       NaN   
                                 17             Aphid                                       NaN   
                                 18             Aphid                                       NaN   
                                 19             Aphid                                       NaN   
                                 20             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        4.0      21             Aphid                                       NaN   
                                 22             Aphid                                       NaN   
                                 23             Aphid                                       NaN   
                                 24             Aphid                                       NaN   
                                 25             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
...                                                                                         ...   
2017-08-25 13:05:14.062 5.0      26             Aphid                                       NaN   
                                 27             Aphid                                       NaN   
                                 28             Aphid                                       NaN   
                                 29             Aphid                                       NaN   
                                 30             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        6.0      31             Aphid                                       NaN   
                                 32             Aphid                                       NaN   
         

In [118]:
idf3 = pandas.merge(idf2, ne2, right_index=True, left_index=True, how='outer')

In [116]:
import itertools

def string_format_range(column_name, bounds):
    lower, upper = bounds
    return [column_name.format(i) for i in range(lower, upper + 1)]

In [120]:
enemy_num_cols = [*string_format_range('e{0}', (1, 9))]
idf3[get_descendant_column_names(get_column_tree(idf2)['fields']['oSets']['oPoints'])+enemy_num_cols]

fields__oSets__oPoints__observations__a1__number  \
datetime                point_id observation_id observation_subject                                                     
2017-08-02 13:12:09.542 0.0      1              Aphid                                                             NaN   
                                 2              Aphid                                                             NaN   
                                 3              Aphid                                                             NaN   
                                 4              Aphid                                                             NaN   
                                 5              Aphid                                                             NaN   
                                 NaN            Natural Enemy                                                     NaN   
                        1.0      6              Aphid                                                             NaN   
                                 7              Aphid                                                             NaN   
                                 8              Aphid                                                             NaN   
                                 9              Aphid                                                             NaN   
                                 10             Aphid                                                             NaN   
                                 NaN            Natural Enemy                                                     NaN   
                        2.0      11             Aphid                                                             NaN   
                                 12             Aphid                                                             NaN   
                                 13             Aphid                                                             NaN   
                                 14             Aphid                                                             NaN   
                                 15             Aphid                                                             NaN   
                                 NaN            Natural Enemy                                                     NaN   
                        3.0      16             Aphid                                                             NaN   
                                 17             Aphid                                                             NaN   
                                 18             Aphid                                                             NaN   
                                 19             Aphid                                                             NaN   
                                 20             Aphid                                                             NaN   
                                 NaN            Natural Enemy                                                     NaN   
                        4.0      21             Aphid                                                             NaN   
                                 22             Aphid                                                             NaN   
                                 23             Aphid                                                             NaN   
                                 24             Aphid                                                             NaN   
                                 25             Aphid                                                             NaN   
                                 NaN            Natural Enemy                                                     NaN   
...                                                                                                               ...   
2017-08-22 15:42:36.395 0.0      1              Aphid                       

In [121]:
idf3.index.names

FrozenList(['datetime', 'point_id', 'observation_id', 'observation_subject'])

In [127]:
sets_columns = get_child_column_names(get_column_tree(idf2)['fields']['oSets']['oPoints']['observations'])
idf3[sets_columns + enemy_num_cols]#.dropna(how='all')

fields__oSets__oPoints__observations__anum  \
datetime                point_id observation_id observation_subject                                               
2017-08-02 13:12:09.542 0.0      1              Aphid                                                       0.0   
                                 2              Aphid                                                       0.0   
                                 3              Aphid                                                       0.0   
                                 4              Aphid                                                       0.0   
                                 5              Aphid                                                       0.0   
                                 NaN            Natural Enemy                                               NaN   
                        1.0      6              Aphid                                                       0.0   
                                 7              Aphid                                                       0.0   
                                 8              Aphid                                                       0.0   
                                 9              Aphid                                                       0.0   
                                 10             Aphid                                                       0.0   
                                 NaN            Natural Enemy                                               NaN   
                        2.0      11             Aphid                                                       0.0   
                                 12             Aphid                                                       0.0   
                                 13             Aphid                                                       0.0   
                                 14             Aphid                                                       0.0   
                                 15             Aphid                                                       0.0   
                                 NaN            Natural Enemy                                               NaN   
                        3.0      16             Aphid                                                       0.0   
                                 17             Aphid                                                       0.0   
                                 18             Aphid                                                       0.0   
                                 19             Aphid                                                       0.0   
                                 20             Aphid                                                       0.0   
                                 NaN            Natural Enemy                                               NaN   
                        4.0      21             Aphid                                                       0.0   
                                 22             Aphid                                                       0.0   
                                 23             Aphid                                                       0.0   
                                 24             Aphid                                                       0.0   
                                 25             Aphid                                                       0.0   
                                 NaN            Natural Enemy                                               NaN   
...                                                                                                         ...   
2017-08-22 15:42:36.395 0.0      1              Aphid                                                       0.0   
                                 2              Aphid                                                       0.0   
                                 3          

In [139]:
display(list(idf.index.names) + idf.columns.tolist())

['datetime',
 'point_id',
 'observation_id',
 'observation_subject',
 'fields__client__displayText',
 'fields__crop',
 'fields__desc',
 'fields__name',
 'fields__oSets__completeSets',
 'fields__oSets__desc',
 'fields__oSets__growthStage',
 'fields__oSets__oPoints__observations__a1__number',
 'fields__oSets__oPoints__observations__a2__number',
 'fields__oSets__oPoints__observations__a3__number',
 'fields__oSets__oPoints__observations__anum',
 'fields__oSets__oPoints__observations__disabled',
 'fields__oSets__oPoints__observations__enum',
 'fields__oSets__obsName',
 'fields__oSets__totalA1',
 'fields__oSets__totalA2',
 'fields__oSets__totalA3',
 'fields__oSets__totalA4',
 'fields__oSets__totalSets']

In [146]:
pandas.options.display.max_columns = 55
pandas.options.display.max_rows = 155
idf3

fields__client__displayText  \
datetime                point_id observation_id observation_subject                               
2017-08-02 13:12:09.542 0.0      1              Aphid                                       NaN   
                                 2              Aphid                                       NaN   
                                 3              Aphid                                       NaN   
                                 4              Aphid                                       NaN   
                                 5              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        1.0      6              Aphid                                       NaN   
                                 7              Aphid                                       NaN   
                                 8              Aphid                                       NaN   
                                 9              Aphid                                       NaN   
                                 10             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        2.0      11             Aphid                                       NaN   
                                 12             Aphid                                       NaN   
                                 13             Aphid                                       NaN   
                                 14             Aphid                                       NaN   
                                 15             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        3.0      16             Aphid                                       NaN   
                                 17             Aphid                                       NaN   
                                 18             Aphid                                       NaN   
                                 19             Aphid                                       NaN   
                                 20             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        4.0      21             Aphid                                       NaN   
                                 22             Aphid                                       NaN   
                                 23             Aphid                                       NaN   
                                 24             Aphid                                       NaN   
                                 25             Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
2017-08-09 09:25:11.710 0.0      1              Aphid                     AAFC SRDC, Tyler Wist   
                                 2              Aphid                                       NaN   
                                 3              Aphid                                       NaN   
                                 4              Aphid                                       NaN   
                                 5              Aphid                                       NaN   
                                 NaN            Natural Enemy                               NaN   
                        1.0      6              Aphid                                       NaN   
                                 7              Aphid                                       NaN   
                                 8              Aphid                                       NaN   
         

In [156]:
sets_columns = get_descendant_column_names(
    get_column_tree(idf3)['fields']['oSets'])
idf3sum = idf3[sets_columns + enemy_num_cols].groupby(
    level=['datetime', 'point_id', 'observation_subject']).sum()

In [149]:
print(visualize_tree(get_column_tree(idf3)))

fields/
.   client/
.   .   displayText : 21
.   crop : 21
.   desc : 15
.   name : 21
.   oSets/
.   .   completeSets : 54
.   .   desc : 0
.   .   growthStage : 54
.   .   oPoints/
.   .   .   observations/
.   .   .   .   a1/
.   .   .   .   .   number : 448
.   .   .   .   a2/
.   .   .   .   .   number : 142
.   .   .   .   a3/
.   .   .   .   .   number : 25
.   .   .   .   anum : 1675
.   .   .   .   disabled : 0
.   .   .   .   enum : 335
.   .   obsName : 54
.   .   totalA1 : 52
.   .   totalA2 : 52
.   .   totalA3 : 52
.   .   totalA4 : 52
.   .   totalSets : 54
e1 : 30
e2 : 21
e3 : 5
e4 : 4
e5 : 4
e6 : 6
e7 : 1
e8 : 87
e9 : 14


In [198]:
idf3sum[enemy_num_cols] = idf3sum[enemy_num_cols].fillna(0)
idf3sum[enemy_num_cols]

e1   e2   e3   e4   e5  \
datetime                point_id observation_subject                            
2017-07-14 12:31:24.194 0.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        1.0  1.0  0.0  0.0  0.0   
                        1.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        2.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        3.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        4.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
2017-07-18 10:31:22.263 0.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        1.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        2.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        3.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        4.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
2017-07-18 10:51:30.195 0.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        1.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        2.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        3.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        4.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
2017-07-18 11:39:46.221 0.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        1.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        2.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        3.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        4.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
2017-07-18 14:25:46.769 0.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        1.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        2.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
                        3.0      Aphid                0.0  0.0  0.0  0.0  0.0   
                                 Natural Enemy        0.0  0.0  0.0  0.0  0.0   
     

In [216]:
idf3sum[idf3sum[enemy_num_cols].sum(axis=1) > 0]

fields__oSets__completeSets  \
datetime                point_id observation_subject                                
2017-07-14 12:31:24.194 0.0      Natural Enemy                                0.0   
2017-07-18 10:51:30.195 0.0      Natural Enemy                                0.0   
2017-07-19 11:22:41.376 0.0      Natural Enemy                                0.0   
2017-07-19 14:07:39.723 2.0      Natural Enemy                                0.0   
                        3.0      Natural Enemy                                0.0   
2017-08-01 11:36:00.012 3.0      Natural Enemy                                0.0   
                        4.0      Natural Enemy                                0.0   
2017-08-01 15:27:40.174 1.0      Natural Enemy                                0.0   
2017-08-01 15:47:22.848 0.0      Natural Enemy                                0.0   
2017-08-02 11:43:20.708 4.0      Natural Enemy                                0.0   
2017-08-03 11:28:09.304 0.0      Natural Enemy                                0.0   
2017-08-04 10:25:17.707 1.0      Natural Enemy                                0.0   
2017-08-09 09:25:11.710 1.0      Natural Enemy                                0.0   
                        2.0      Natural Enemy                                0.0   
                        3.0      Natural Enemy                                0.0   
                        4.0      Natural Enemy                                0.0   
2017-08-09 10:06:25.480 0.0      Natural Enemy                                0.0   
                        1.0      Natural Enemy                                0.0   
2017-08-10 14:37:29.806 0.0      Natural Enemy                                0.0   
                        1.0      Natural Enemy                                0.0   
                        2.0      Natural Enemy                                0.0   
                        3.0      Natural Enemy                                0.0   
                        4.0      Natural Enemy                                0.0   
2017-08-17 11:12:02.820 0.0      Natural Enemy                                0.0   
                        1.0      Natural Enemy                                0.0   
                        2.0      Natural Enemy                                0.0   
                        3.0      Natural Enemy                                0.0   
                        4.0      Natural Enemy                                0.0   
                        5.0      Natural Enemy                                0.0   
                        6.0      Natural Enemy                                0.0   
                        7.0      Natural Enemy                                0.0   
                        8.0      Natural Enemy                                0.0   
                        9.0      Natural Enemy                                0.0   
2017-08-17 13:06:30.183 0.0      Natural Enemy                                0.0   
                        1.0      Natural Enemy                                0.0   
                        3.0      Natural Enemy                                0.0   
                        4.0      Natural Enemy                                0.0   
                        5.0      Natural Enemy                                0.0   
                        6.0      Natural Enemy                                0.0   
                        8.0      Natural Enemy                                0.0   
                        9.0      Natural Enemy                                0.0   
2017-08-18 13:33:03.584 0.0      Natural Enemy                                0.0   
                        1.0      Natural Enemy                                0.0   
                        2.0      Natural Enemy                                0.0   
                        7.0      Natural Enemy                                0.0   
2017-08-18 14:16:58.657 0.0      Natural Enemy             